In [9]:
import requests
import mysql.connector
import time
import discord_notifier

def db_connect():
    return mysql.connector.connect(user='lol_user', password='your_password',
                                   host='35.203.46.85', database='lol_retention')

def retrieve_puuids(increase_pool=False):
    cnx = db_connect()
    cursor = cnx.cursor()
    if increase_pool:
        # Retrieve PUUIDs for increasing the match pool for each player
        query = "SELECT puuid FROM summoner_details"
    else:
        # Retrieve PUUIDs for players who don't have any match id
        query = """
        SELECT sd.puuid 
        FROM summoner_details sd 
        LEFT JOIN match_ids mi ON sd.puuid = mi.puuid 
        WHERE mi.puuid IS NULL
        """
    
    cursor.execute(query)
    puuids = [row[0] for row in cursor.fetchall()]
    cursor.close()
    cnx.close()
    return puuids

def insert_match_ids(match_id_puuid_pairs):
    cnx = db_connect()
    cursor = cnx.cursor()
    try:
        add_match_ids = """
        INSERT INTO match_ids (match_id, puuid)
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE
        match_id = VALUES(match_id), puuid = VALUES(puuid)
        """
        cursor.executemany(add_match_ids, match_id_puuid_pairs)
        cnx.commit()
        print(f"Inserted {len(match_id_puuid_pairs)} match ID-PUUID pairs.")

    except mysql.connector.Error as err:
        print("Error:", err)
    finally:
        cursor.close()
        cnx.close()

def pull_match_ids(api_key, puuid, base_url, matches_per_player):
    url = f"{base_url}{puuid}/ids?start=0&count={matches_per_player}&api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return [(match_id, puuid) for match_id in response.json()]
    else:
        print(f"Error {response.status_code} fetching matches for {puuid}: {response.text}")
        return []

def manage_api_rate_limit():
    time.sleep(1.3)  # Adjust based on the Riot Games API rate limits

def main(increase_pool=False, matches_per_player=1):
    api_key = "RGAPI-a6eedfcb-600e-4f76-b541-0a25e232aa36"
    base_url = "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/"
    players = retrieve_puuids(increase_pool)
    batch_size = 200  # Set the threshold for batch processing
    accumulated_pairs = []

    try:
        count = 1
        total = len(players)
        for puuid in players:
            print(f"Fetching: {count} out of {total} - PUUID: {puuid}")
            match_id_puuid_pairs = pull_match_ids(api_key, puuid, base_url, matches_per_player)
            if match_id_puuid_pairs:
                accumulated_pairs.extend(match_id_puuid_pairs)
                if len(accumulated_pairs) >= batch_size:
                    insert_match_ids(accumulated_pairs)
                    accumulated_pairs = []  # Reset the list after insertion
            manage_api_rate_limit()
            count += 1
        
        # Insert any remaining pairs that didn't reach the batch size
        if accumulated_pairs:
            insert_match_ids(accumulated_pairs)

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        discord_notifier.send_discord_message()

if __name__ == "__main__":
    main(increase_pool=False, matches_per_player=1)  # Set True if you want to increase the match pool for each player and define the amount of matches

Fetching: 1 out of 5603 - PUUID: -PCVEbUpc75rsjslX00Wy03xmEP9cvl_u2wbgrtZ8LM7Y7QXlZ15G8-hUc5svGH8DNRJO3LZeosJoA
Fetching: 2 out of 5603 - PUUID: 0xdfJQfJU6t76H10GHTCMyG6MvQtC3pW5DiUln-j14gA9Rx1PYXIwPxOHxYeuUqkIKXA2P9eCz7Wlw
Fetching: 3 out of 5603 - PUUID: 2HpW_IzVXG7ShW6XjwQdcdC5bWdcGb-A88Q5yVjICmHE98GcGSf6-52VW_zMVV_GTs4grY0bv88c8w
Fetching: 4 out of 5603 - PUUID: 6IKvecWQSFWtLRFCun-sNWCkYzPHe-5EPis7DS5WSMdnf0s9jrmQwEJvPue_N4zvzcowdCe7Im7MEg
Fetching: 5 out of 5603 - PUUID: ayJsgL7XGYc66QlrYHuTCI3gyE9rtQ1w6lZOjeYR35NV9wZ2lLxWQ7i9TYMuZzH7gARkQwUPnZWq-A
Fetching: 6 out of 5603 - PUUID: bpBeMohKhtZ2iy4r7lhqQNPrk0jGcBEWrm8D0QSKj3fdtxrcg-jnntFSn2CpU7eSGXUh_ywr8iHB7g
Fetching: 7 out of 5603 - PUUID: DgI9Q0wbOYhvfAAViR9qffRhTCe8HdW4fL2v_HQ1TAcHrncDPI2eQ-Dc4VLma79j2nFYUkDC_3F1Fw
Fetching: 8 out of 5603 - PUUID: DGJIRbsWDxEyRfjcdCzfZmnKxxRMG-wvGGPLb9e7eZLjoX2W2zfQzOm8qtO-Id2eufNTKjqzAWs9TQ
Fetching: 9 out of 5603 - PUUID: dgNA5mwiuVJMwoegoqmrswgAZH2EM9sWSaHwPweu9gdzT0CpJy4ngz2_Ybgb9tLp_JXY0ao

MissingSchema: Invalid URL 'Completed pulling and inserting match IDs.': No scheme supplied. Perhaps you meant https://Completed pulling and inserting match IDs.?